In [1]:
# hyperparameter tuning

# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os
import dagshub
import mlflow

mlflow.set_tracking_uri('https://dagshub.com/avi350751/mlops-new-mini-project.mlflow')
dagshub.init(repo_owner='avi350751', repo_name='mlops-new-mini-project', mlflow=True)


# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])


# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

# Normalize the text data
df = normalize_text(df)

x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the experiment name
mlflow.set_experiment("LoR Hyperparameter Tuning")

# Define hyperparameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

# Start the parent run for hyperparameter tuning
with mlflow.start_run():

    # Perform grid search
    grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Log each parameter combination as a child run
    for params, mean_score, std_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['std_test_score']):
        with mlflow.start_run(run_name=f"LR with params: {params}", nested=True):
            model = LogisticRegression(**params)
            model.fit(X_train, y_train)
            
            # Model evaluation
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            
            # Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metric("mean_cv_score", mean_score)
            mlflow.log_metric("std_cv_score", std_score)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            
            
            # Print the results for verification
            print(f"Mean CV Score: {mean_score}, Std CV Score: {std_score}")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1}")

    # Log the best run details in the parent run
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    mlflow.log_params(best_params)
    mlflow.log_metric("best_f1_score", best_score)
    
    print(f"Best Params: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save and log the notebook
    import os
    notebook_path = "exp3_lor_hp.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    # Log model
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")

Accessing as avi350751

Initialized MLflow to track repo "avi350751/mlops-new-mini-project"

Repository avi350751/mlops-new-mini-project initialized!

/var/folders/1p/1srh72nx48n3v99r1_jr_1pm0000gn/T/ipykernel_39108/281815890.py:85: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
2024/08/11 23:29:13 INFO mlflow.tracking.fluent: Experiment with name 'LoR Hyperparameter Tuning' does not exist. Creating a new experiment.
2024/08/11 23:29:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'} at: https://dagshub.com/avi350751/mlops-new-mini-project.mlflow/#/experiments/2/runs/a96658bd81ce45b0b6e872a134cdf0a0.
2024/08/11 23:29:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/avi350751/mlops-new-mini-project.mlflow/#/experiments/

Mean CV Score: 0.7051163373017877, Std CV Score: 0.014297013862890536
Accuracy: 0.739277108433735
Precision: 0.7749419953596288
Recall: 0.6581280788177339
F1 Score: 0.7117741076185402


...

2024/08/11 23:29:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'} at: https://dagshub.com/avi350751/mlops-new-mini-project.mlflow/#/experiments/2/runs/b84ec51fddc347b59527ccbaf3adfa3b.
2024/08/11 23:29:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/avi350751/mlops-new-mini-project.mlflow/#/experiments/2.


Mean CV Score: 0.7712238577695444, Std CV Score: 0.011375181888880538
Accuracy: 0.7812048192771084
Precision: 0.7774480712166172
Recall: 0.774384236453202
F1 Score: 0.7759131293188549


2024/08/11 23:29:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run LR with params: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'} at: https://dagshub.com/avi350751/mlops-new-mini-project.mlflow/#/experiments/2/runs/e21fd3b0fb3e424e90a83bef00e9fbc2.
2024/08/11 23:29:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/avi350751/mlops-new-mini-project.mlflow/#/experiments/2.


Mean CV Score: 0.781002998936019, Std CV Score: 0.005279351561572635
Accuracy: 0.7836144578313253
Precision: 0.7715930902111324
Recall: 0.7921182266009852
F1 Score: 0.7817209528439475
Best Params: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Best F1 Score: 0.7901458706737883


[NbConvertApp] Converting notebook exp3_lor_hp.ipynb to notebook
Traceback (most recent call last):
  File "/Users/avi/Desktop/CampusX/MLOPS/mlops-new-mini-project/myenv/lib/python3.12/site-packages/nbformat/reader.py", line 19, in parse_json
    nb_dict = json.loads(s, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from 